#### Transformers

attention mechanism: a model for processing text will (i) use parameter
sharing to cope with long input passages of differing lengths and (ii) contain connections
between word representations that depend on the words themselves. The transformer
acquires both properties by using **dot-product self-attention**.

the dot product attention based mechanism:
$$\mathrm{sa}_n[X_1, \dots, X_N] = \sum_{m=1}^{N} a[X_m, X_n] V_m$$

The scalar weight $a[x_m,x_n]$ is the attention that the nth output pays to input $x_m$. The $N$
weights $a[\bullet,x_n]$ are non-negative and sum to one. Hence, self-attention can be thought
of as routing the values in different proportions to create each output

l 
